In [ ]:
# Dynamic Pricing for Urban Parking Lots
# Capstone Project - Summer Analytics 2025



# 1. Objective & Justification

# This project aims to dynamically price urban parking spots based on real-time data. Traditional fixed-rate pricing creates inefficiencies — either overutilization or underuse.
# Dynamic pricing improves usage, reduces congestion, and increases revenue by adapting to real-world conditions such as traffic, occupancy, and vehicle types.


# 2. Dataset Overview

# The dataset contains:
# - 14 parking lots
# - 18,368 records over 73 days
# - Features include: timestamp, occupancy, capacity, queue length, traffic condition, special day indicator, vehicle type, and coordinates


In [1]:
# 3. Setup and Data Load

!pip install geopy bokeh --quiet
!pip install pathway bokeh --quiet
import pandas as pd
import numpy as np
import pathway as pw
from geopy.distance import geodesic
from bokeh.plotting import figure, show, output_notebook
from bokeh.io import push_notebook
output_notebook()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# 4.Upload  dataset

from google.colab import files
uploaded = files.upload()
df = pd.read_csv('dataset.csv')

Saving dataset.csv to dataset.csv


In [3]:
# 5. Preprocessing

df['timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], format='%d-%m-%Y %H:%M:%S')
df = df.sort_values(['SystemCodeNumber', 'timestamp'])

In [8]:
# 6. Model 1 - Baseline Linear Model

BASE_PRICE = 10
ALPHA = 2
df_model1 = df.copy()
df_model1['price'] = BASE_PRICE
grouped = df_model1.groupby('SystemCodeNumber')
for name, group in grouped:
    price_list = [BASE_PRICE]
    for i in range(1, len(group)):
        occ = group.iloc[i]['Occupancy']
        cap = group.iloc[i]['Capacity']
        price_list.append(price_list[-1] + ALPHA * (occ / cap))
    df_model1.loc[group.index, 'price'] = price_list
print(df_model1.head())

/tmp/ipython-input-8-3366439836.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[10, np.float64(10.221837088388215), np.float64(10.499133448873483), np.float64(10.87001733102253), np.float64(11.389948006932407), np.float64(12.003466204506063), np.float64(12.762564991334486), np.float64(13.618717504332754), np.float64(14.516464471403811), np.float64(15.438474870017329), np.float64(16.370883882149045), np.float64(17.282495667244365), np.float64(18.107452339688038), np.float64(18.8526863084922), np.float64(19.518197573656845), np.float64(20.090121317157713), np.float64(20.651646447140383), np.float64(21.1473136915078), np.float64(21.334488734835357), np.float64(21.538994800693242), np.float64(21.785095320623917), np.float64(22.07279029462738), np.float64(22.467937608318888), np.float64(22.911611785095317), np.float64(23.424610051993064), np.float64(23.986135181975733), np.float64(24.603119584055456), n

   ID SystemCodeNumber  Capacity   Latitude  Longitude  Occupancy VehicleType  \
0   0      BHMBCCMKT01       577  26.144536  91.736172         61         car   
1   1      BHMBCCMKT01       577  26.144536  91.736172         64         car   
2   2      BHMBCCMKT01       577  26.144536  91.736172         80         car   
3   3      BHMBCCMKT01       577  26.144536  91.736172        107         car   
4   4      BHMBCCMKT01       577  26.144536  91.736172        150        bike   

  TrafficConditionNearby  QueueLength  IsSpecialDay LastUpdatedDate  \
0                    low            1             0      04-10-2016   
1                    low            1             0      04-10-2016   
2                    low            2             0      04-10-2016   
3                    low            2             0      04-10-2016   
4                    low            2             0      04-10-2016   

  LastUpdatedTime           timestamp      price  
0        07:59:00 2016-10-04 07:59:

In [9]:
# 7. Model 2 - Demand-Based Model

df_model2 = df.copy()
vehicle_weights = {'car': 1.0, 'bike': 0.5, 'truck': 1.5}
df_model2['vehicle_weight'] = df_model2['VehicleType'].map(vehicle_weights)
df_model2['traffic_level'] = df_model2['TrafficConditionNearby'].map({'low': 1, 'medium': 2, 'high': 3})
def normalize(s): return (s - s.min()) / (s.max() - s.min())
df_model2['occupancy_norm'] = normalize(df_model2['Occupancy'] / df_model2['Capacity'])
df_model2['queue_norm'] = normalize(df_model2['QueueLength'])
df_model2['traffic_norm'] = normalize(df_model2['traffic_level'].fillna(1))
df_model2['vehicle_weight_norm'] = normalize(df_model2['vehicle_weight'])
alpha, beta, gamma, delta, epsilon, lambda_ = 1, 0.5, 0.3, 1, 1, 0.8
df_model2['demand'] = (
    alpha * df_model2['occupancy_norm'] +
    beta * df_model2['queue_norm'] -
    gamma * df_model2['traffic_norm'] +
    delta * df_model2['IsSpecialDay'] +
    epsilon * df_model2['vehicle_weight_norm']
)
df_model2['demand_norm'] = normalize(df_model2['demand'])
df_model2['price'] = BASE_PRICE * (1 + lambda_ * df_model2['demand_norm'])
df_model2['price'] = df_model2['price'].clip(5, 20)
print(df_model2.head())

   ID SystemCodeNumber  Capacity   Latitude  Longitude  Occupancy VehicleType  \
0   0      BHMBCCMKT01       577  26.144536  91.736172         61         car   
1   1      BHMBCCMKT01       577  26.144536  91.736172         64         car   
2   2      BHMBCCMKT01       577  26.144536  91.736172         80         car   
3   3      BHMBCCMKT01       577  26.144536  91.736172        107         car   
4   4      BHMBCCMKT01       577  26.144536  91.736172        150        bike   

  TrafficConditionNearby  QueueLength  IsSpecialDay  ...           timestamp  \
0                    low            1             0  ... 2016-10-04 07:59:00   
1                    low            1             0  ... 2016-10-04 08:25:00   
2                    low            2             0  ... 2016-10-04 08:59:00   
3                    low            2             0  ... 2016-10-04 09:32:00   
4                    low            2             0  ... 2016-10-04 09:59:00   

  vehicle_weight traffic_level  

In [26]:
# 8. Visualization
# =============================
def plot_price_evolution(df, model_name, lot_id):
    df_lot = df[df['SystemCodeNumber'] == lot_id].sort_values('timestamp')
    p = figure(title=f"{model_name} Pricing - Lot {lot_id}", x_axis_type="datetime", width=800)
    p.line(df_lot['timestamp'], df_lot['price'], legend_label="Price ($)", line_width=2, color='navy')
    p.xaxis.axis_label = 'Time'
    p.yaxis.axis_label = 'Price'
    show(p, notebook_handle=True)
    # plot for one lot
plot_price_evolution(df_model1, "Model 1 - Linear", "BHMBCCMKT01")
plot_price_evolution(df_model2, "Model 2 - Demand-Based", "BHMBCCMKT01")


In [24]:
# 9. Export Results

df_model3.to_csv("final_dynamic_prices.csv", index=False)
print(" All models executed and final prices saved to final_dynamic_prices.csv")

✅ All models executed and final prices saved to final_dynamic_prices.csv
